In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
dnevi=pd.read_csv('data/po_dnevih.csv', sep=";")#
udelezenci=pd.read_csv('data/po_vrstah_udelezencev.csv', sep=";")
ceste=pd.read_csv('data/po_vrsti_ceste.csv', sep=";")
meseci=pd.read_csv('data/po_mesecih.csv', sep=";")
regije=pd.read_csv('data/po_regijah.csv', sep=";")
lokacije=pd.read_csv('data/po_lokaciji.csv', sep=";")
ura=pd.read_csv('data/po_urah.csv', sep=";")
alkohol=pd.read_csv('data/alkohol.csv', sep=";")

In [4]:
alkohol["Stopnje alkoholiziranosti 0,80 in več"].fillna(alkohol.groupby("Kategorija")["Stopnje alkoholiziranosti 0,80 in več"].transform("mean"), inplace=True)
alkohol["Stopnje alkoholiziranosti 0,80 in več"] = alkohol["Stopnje alkoholiziranosti 0,80 in več"].astype(int)

In [5]:
# 
dnevi_sum = dnevi.groupby(['Dan v tednu'], as_index = False).agg({'Prometne nesreče s smrtnim izidom': [np.sum],
                                                                  'Prometne nesreče s telesno poškodbo': [np.sum],
                                                                  'Prometne nesreče z materialno škodo': [np.sum],
                                                                  'Prometne nesreče - SKUPAJ': [np.sum]
                                                                 }) 

dnevi_sum.columns = ["dan_v_tednu", "st_smrt", "st_telesna_poskodba", "st_materialna_skoda", "skupaj"]    
dnevi_sum = dnevi_sum.sort_values(by="skupaj", ascending=[False])
dnevi_sum;

In [6]:
# draws plot
trace1 = go.Bar(
    y=dnevi_sum["dan_v_tednu"],
    x=dnevi_sum["st_smrt"],
    name='Število nesreč s smrtnim izidom',
    orientation = "h"
)
trace2 = go.Bar(
    y=dnevi_sum["dan_v_tednu"],
    x=dnevi_sum["st_telesna_poskodba"],
    name='Število nesreč telesnih poškodb',
    orientation = "h"
)
trace3 = go.Bar(
    y=dnevi_sum["dan_v_tednu"],
    x=dnevi_sum["st_materialna_skoda"],
    name='Število nesreč z materialno škodo',
    orientation = "h"
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

In [7]:
# 
smrti_sum = dnevi.groupby(['leto'], as_index = False).agg({'Prometne nesreče s smrtnim izidom': [np.sum],
                                                                  'Prometne nesreče s telesno poškodbo': [np.sum],
                                                                  'Prometne nesreče z materialno škodo': [np.sum],
                                                                  'Prometne nesreče - SKUPAJ': [np.sum]
                                                                 }) 

smrti_sum.columns = ["leto", "st_smrt", "st_telesna_poskodba", "st_materialna_skoda", "skupaj"]    
smrti_sum = smrti_sum.sort_values(by="leto", ascending=[False])
smrti_sum;

In [8]:
# Create a trace
trace = go.Scatter(
    x = smrti_sum["leto"],
    y = smrti_sum["st_smrt"]
)

data = [trace]
iplot(data, filename='basic-line')

In [9]:
mapbox_access_token = "pk.eyJ1IjoiaW1pbG9zayIsImEiOiJjamZ2M3c2MmQwcHk5Mndtd3lyMTI2bHJqIn0.FMLgB-WENz3ZSe1gRWRJHg"

data = go.Data([
    go.Scattermapbox(
        lat=['46.1512'],
        lon=['14.9955'],
        mode='markers',
    )
])
layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://gist.githubusercontent.com/imilosk/47013ab15f8a329b0497853f95d081d5/raw/48d364de4c88b0fc6fbd9e42012b47f87def285d/slovenia.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=46.1512,
            lon=14.9955
        ),
        pitch=0,
        zoom=7.0,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
#iplot(fig, filename='county-level-choropleths-python');

In [10]:
# 
alko_sum = alkohol.groupby(['Kategorija'], as_index = False).agg({'Stopnje alkoholiziranosti 0,50 in več': [np.sum],
                                                                  'Stopnje alkoholiziranosti 0,80 in več': [np.sum],
                                                                  'Stopnje alkoholiziranosti 1,50 in več': [np.sum],
                                                                  'Število alkoholiziranih udeležencev - SKUPAJ': [np.sum]
                                                                 }) 

alko_sum.columns = ["kategorija","alko_05", "alko_08", "alko_15", "skupaj"]
alko_sum = alko_sum.sort_values(by="skupaj", ascending=[False])
alko_sum = alko_sum.drop([0, 3, 9])

In [11]:
trace1 = go.Bar(
    x=alko_sum["kategorija"],
    y=alko_sum["alko_05"],
    name='Stopnje alkoholiziranosti 0,50 in več'
)
trace2 = go.Bar(
    x=alko_sum["kategorija"],
    y=alko_sum["alko_08"],
    name='Stopnje alkoholiziranosti 0,80 in več'
)
trace3 = go.Bar(
    x=alko_sum["kategorija"],
    y=alko_sum["alko_15"],
    name='Stopnje alkoholiziranosti 1,50 in več'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')